# Reframe Package Example Notebook

This notebook demonstrates the core functionality of the Reframe package for analyzing and rewriting drug-related and stigmatizing content.

## Features demonstrated:
1. Classifying drug-related content
2. Identifying stigmatizing language
3. Analyzing text style
4. Rewriting stigmatizing content

In [4]:
# Import the destigmatizer package and its components
from destigmatizer import core
from destigmatizer.clients import get_client
from destigmatizer.utils import get_default_model, get_model_mapping

# Define example texts for demonstration
NON_DRUG_POST = "I think we should really work on the housing crisis in urban areas. The homeless are getting scary."
DRUG_NON_STIGMA_POST = "I really feel for people who suffer from substance use disorder, being unable to control an impulse due to dependency sounds scary."
DRUG_AND_STIGMA_POST = "Addicts really need to get control of themselves. Just stop doing drugs. It seems like these people are just lacking willpower."

## Setup API Client

First, we need to initialize the client with your API key. This example uses OpenAI, but you could use other clients as supported by the package.

In [5]:
# Initialize with your API key
# You can either load from a secrets file or directly input your key

import json
import os

# Option 1: Load from secrets file
try:
    with open('./secrets.json') as f:
        secrets = json.load(f)
    api_key = secrets.get('OPENAI_API_KEY')
except (FileNotFoundError, json.JSONDecodeError, KeyError):
    # Option 2: Get from environment variable
    api_key = os.environ.get('OPENAI_API_KEY')

# Initialize client
client = get_client('openai', api_key=api_key)
# model = get_default_model('openai')  # Get default model for this client type
model = "gpt-4o-2024-11-20"
print(f"Client initialized with model: {model}")

Client initialized with model: gpt-4o-2024-11-20


In [6]:
# # Initialize with your API key
# # You can either load from a secrets file or directly input your key

# import json
# import os

# # Option 1: Load from secrets file
# try:
#     with open('./secrets.json') as f:
#         secrets = json.load(f)
#     api_key = secrets.get('TOGETHER_API_KEY')
# except (FileNotFoundError, json.JSONDecodeError, KeyError):
#     # Option 2: Get from environment variable
#     api_key = os.environ.get('TOGETHER_API_KEY')

# # Initialize client
# client = get_client('together', api_key=api_key)
# model = get_model_mapping("medium", 'together')  # Get default model for this client type
# print(f"Client initialized with model: {model}")

In [7]:
# # Initialize with your API key
# # You can either load from a secrets file or directly input your key

# import json
# import os

# # Option 1: Load from secrets file
# try:
#     with open('./secrets.json') as f:
#         secrets = json.load(f)
#     api_key = secrets.get('ANTHROPIC_API_KEY')
# except (FileNotFoundError, json.JSONDecodeError, KeyError):
#     # Option 2: Get from environment variable
#     api_key = os.environ.get('ANTHROPIC_API_KEY')

# # Initialize client
# client = get_client('claude', api_key=api_key)
# model = get_model_mapping("medium", 'claude')  # Get default model for this client type
# print(f"Client initialized with model: {model}")

## Configuration Management

In [8]:
import os
from destigmatizer.config_manager import get_user_config_path, create_default_config, load_config, add_named_config, save_config, display_config

# Get the recommended location for user configuration
user_config_path = get_user_config_path()
print(f"Recommended user config path: {user_config_path}")

# Create a default configuration if it doesn't exist
config_example_path = "./reframe_example_config.json"
if not os.path.exists(config_example_path):
    print("\nCreating example configuration file...")
    create_default_config(config_example_path, overwrite=True)

# Load the configuration
print("\nLoading configuration...")
config = load_config(config_example_path)

# Display the current configuration
print("\nCurrent configuration:")
display_config(config)

# Add a custom configuration for creative writing
print("\nAdding custom configuration...")
config = add_named_config(
    config,
    name="highly_creative",
    model_name="medium",
    temperature=0.9,
    max_tokens=2500,
    top_p=0.95,
    frequency_penalty=0.2
)

# Save the modified configuration
save_config(config, config_example_path)

# Display the updated configuration
print("\nUpdated configuration:")
display_config(config["named_configs"]["highly_creative"])

Recommended user config path: /Users/guomuqi/.reframe/config.json

Loading configuration...

Current configuration:
default_config_name: medium_quality
model_mappings:
  small:
    openai: gpt-4o-mini
    together: meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo
    claude: claude-3-haiku-20241022
    ollama: llama3:8b
    gemini: gemini-1.0-pro-001
  medium:
    openai: gpt-4o
    together: meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo
    claude: claude-3-5-sonnet-20240620
    ollama: llama3:70b
    gemini: gemini-1.5-pro-001
  large:
    openai: gpt-4o-2024-05-13
    together: mistralai/Mixtral-8x22B-Instruct-v0.1
    claude: claude-3-opus-20240229
    ollama: mixtral
    gemini: gemini-1.5-flash-001
default_models:
  openai: gpt-4o
  together: meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo
  claude: claude-3-5-haiku-20241022
  ollama: llama3:8b
  gemini: gemini-1.0-pro-001
named_configs:
  high_quality:
    model_name: large
    temperature: 0.0
    max_tokens: 4000
    top_p: 1.0
  medium

## Example Texts

Let's take a look at our example texts:

In [9]:
# Display the example texts
print("NON_DRUG_POST:")
print(NON_DRUG_POST)
print("\nDRUG_NON_STIGMA_POST:")
print(DRUG_NON_STIGMA_POST)
print("\nDRUG_AND_STIGMA_POST:")
print(DRUG_AND_STIGMA_POST)

NON_DRUG_POST:
I think we should really work on the housing crisis in urban areas. The homeless are getting scary.

DRUG_NON_STIGMA_POST:
I really feel for people who suffer from substance use disorder, being unable to control an impulse due to dependency sounds scary.

DRUG_AND_STIGMA_POST:
Addicts really need to get control of themselves. Just stop doing drugs. It seems like these people are just lacking willpower.


## 1. Classifying Drug-Related Content

Let's classify texts to determine if they're related to drugs or substance use.

In [10]:
# Classify the non-drug post
print(f"Classifying drug content for: \"{NON_DRUG_POST[:50]}...\"")
non_drug_result = core.classify_if_drug(text=NON_DRUG_POST, client=client, model=model)
print(f"Drug classification result: {non_drug_result}")

# Classify the drug and stigma post
print(f"\nClassifying drug content for: \"{DRUG_AND_STIGMA_POST[:50]}...\"")
drug_stigma_result = core.classify_if_drug(text=DRUG_AND_STIGMA_POST, client=client, model=model)
print(f"Drug classification result: {drug_stigma_result}")

# Classify the drug non-stigma post
print(f"\nClassifying drug content for: \"{DRUG_NON_STIGMA_POST[:50]}...\"")
drug_non_stigma_result = core.classify_if_drug(text=DRUG_NON_STIGMA_POST, client=client, model=model)
print(f"Drug classification result: {drug_non_stigma_result}")

Classifying drug content for: "I think we should really work on the housing crisi..."
Drug classification result: nd

Classifying drug content for: "Addicts really need to get control of themselves. ..."
Drug classification result: d

Classifying drug content for: "I really feel for people who suffer from substance..."
Drug classification result: nd


## 2. Identifying Stigmatizing Language

Next, let's identify if texts contain stigmatizing language.

In [11]:
# Classify the drug non-stigma post
print(f"Classifying stigmatizing language for: \"{DRUG_NON_STIGMA_POST[:50]}...\"")
non_stigma_result = core.classify_if_stigma(text=DRUG_NON_STIGMA_POST, client=client, model=model)
print(f"Stigma classification result: {non_stigma_result}")

# Classify the drug and stigma post
print(f"\nClassifying stigmatizing language for: \"{DRUG_AND_STIGMA_POST[:50]}...\"")
stigma_result = core.classify_if_stigma(text=DRUG_AND_STIGMA_POST, client=client, model=model)
print(f"Stigma classification result: {stigma_result}")

# Classify the non-drug post
print(f"\nClassifying stigmatizing language for: \"{NON_DRUG_POST[:50]}...\"")
non_drug_stigma_result = core.classify_if_stigma(text=NON_DRUG_POST, client=client, model=model)
print(f"Stigma classification result: {non_drug_stigma_result}")

Classifying stigmatizing language for: "I really feel for people who suffer from substance..."
Stigma classification result: ns

Classifying stigmatizing language for: "Addicts really need to get control of themselves. ..."
Stigma classification result: s, labeling: uses the term 'addicts,' which is stigmatizing and reduces individuals to their substance use, stereotyping: assumes that people with addiction lack willpower and oversimplifies the complexity of addiction, separation: implies a divide between people who use drugs and those who are perceived as having self-control, discrimination: suggests that people with addiction are at fault for their condition, potentially justifying a lack of support or empathy.

Classifying stigmatizing language for: "I think we should really work on the housing crisi..."
Stigma classification result: s, labeling: refers to homeless individuals in a generalized and dehumanizing way, stereotyping: implies that homeless individuals are inherently "scar

## 3. Combined Classification

Now let's classify for both drug content and stigmatizing language together.

In [12]:
# Perform combined classification on the drug and stigma post
print(f"Analyzing text for drug content and stigma: \"{DRUG_AND_STIGMA_POST[:50]}...\"")

drug_result = core.classify_if_drug(text=DRUG_AND_STIGMA_POST, client=client, model=model)
stigma_result = core.classify_if_stigma(text=DRUG_AND_STIGMA_POST, client=client, model=model)

print(f"Drug classification: {drug_result}")
print(f"Stigma classification: {stigma_result}")

# Let's analyze the non-stigma post
print(f"\nAnalyzing text for drug content and stigma: \"{DRUG_NON_STIGMA_POST[:50]}...\"")
drug_result2 = core.classify_if_drug(text=DRUG_NON_STIGMA_POST, client=client, model=model)
stigma_result2 = core.classify_if_stigma(text=DRUG_NON_STIGMA_POST, client=client, model=model)
print(f"Drug classification: {drug_result2}")
print(f"Stigma classification: {stigma_result2}")

# Let's analyze the non-drug post
print(f"\nAnalyzing text for drug content and stigma: \"{NON_DRUG_POST[:50]}...\"")
drug_result3 = core.classify_if_drug(text=NON_DRUG_POST, client=client, model=model)
stigma_result3 = core.classify_if_stigma(text=NON_DRUG_POST, client=client, model=model)
print(f"Drug classification: {drug_result3}")
print(f"Stigma classification: {stigma_result3}")
# print(f"Combined result: ({drug_result3}, {stigma_result3})")

Analyzing text for drug content and stigma: "Addicts really need to get control of themselves. ..."
Drug classification: d
Stigma classification: s, labeling: uses the term 'addicts,' which is stigmatizing and reduces individuals to their substance use, stereotyping: assumes that people with addiction lack willpower and oversimplifies the complexity of addiction, separation: implies a divide between people who use drugs and those who are perceived as having self-control, discrimination: suggests that people with addiction are at fault for their condition, potentially justifying a lack of support or empathy.

Analyzing text for drug content and stigma: "I really feel for people who suffer from substance..."
Drug classification: nd
Stigma classification: ns

Analyzing text for drug content and stigma: "I think we should really work on the housing crisi..."
Drug classification: nd
Stigma classification: s, labeling: refers to homeless individuals in a generalized and dehumanizing way, ste

## 4. Text Style Analysis

Before rewriting, we can analyze the style of the text.

In [13]:
# Analyze the style of the example stigmatizing post
example_style = core.analyze_text_llm(text=DRUG_AND_STIGMA_POST, client=client, model=model)
print(f"Text style analysis for example stigmatizing post:\n{example_style}")

# Also analyze the non-stigma post
non_stigma_style = core.analyze_text_llm(text=DRUG_NON_STIGMA_POST, client=client, model=model)
print(f"\nText style analysis for non-stigmatizing post:\n{non_stigma_style}")

Text style analysis for example stigmatizing post:
{'punctuation_usage': 'moderate, with . being most frequent', 'passive_voice_usage': 'none', 'sentence_length_variation': 'ranging from short (4 words) to long (9 words) with an average of 7.0 words per sentence', 'lexical_diversity': '123.48 (MTLD)', 'top_emotions': 'judgmental'}

Text style analysis for non-stigmatizing post:
{'punctuation_usage': 'moderate, with ,, . being most frequent', 'passive_voice_usage': 'none', 'sentence_length_variation': 'ranging from short (22 words) to long (22 words) with an average of 22.0 words per sentence', 'lexical_diversity': '135.52 (MTLD)', 'top_emotions': 'compassion'}


## 5. Rewriting Stigmatizing Content

Let's rewrite stigmatizing content to be more neutral.

In [14]:
# Get stigma classification for the example post
example_post = DRUG_AND_STIGMA_POST
example_stigma = core.classify_if_stigma(text=example_post, client=client, model=model)

# Get text style for rewriting
example_style_instruct = str(core.analyze_text_llm(text=example_post, client=client, model=model))

# Rewrite the stigmatizing content
rewritten_example = core.rewrite_to_destigma(
    text=example_post,
    explanation=example_stigma,
    style_instruct=example_style_instruct,
    model=model,
    client=client
)

print(f"Original: {example_post}")
print(f"Rewritten: {rewritten_example}")

Original: Addicts really need to get control of themselves. Just stop doing drugs. It seems like these people are just lacking willpower.
Rewritten: people struggling with substance use face complex challenges. recovery requires support, understanding, and access to resources.


## 6. Workflow Encapsulation

Finally, you can also see how all the functions work together in the pipeline

In [15]:
rewrote_text = core.analyze_and_rewrite_text(DRUG_AND_STIGMA_POST, client, model)
print(f"Original: {DRUG_AND_STIGMA_POST}")
print(f"Rewritten: {rewrote_text}")

Step 1: Classifying drug-related content...
Text is not drug-related. Skipping further analysis.
Original: Addicts really need to get control of themselves. Just stop doing drugs. It seems like these people are just lacking willpower.
Rewritten: Addicts really need to get control of themselves. Just stop doing drugs. It seems like these people are just lacking willpower.


## Summary

In this notebook, we've demonstrated the core functionality of the Reframe package:
1. Classifying drug-related content
2. Identifying stigmatizing language
3. Analyzing text style
4. Rewriting stigmatizing content to be more neutral while preserving the original message

These tools can be used to analyze and improve communication about substance use disorders and reduce harmful stigma.